# PNAD COVID-19

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit, col, when

import matplotlib.pyplot as plt
import seaborn as sns

## Conectando com Spark

In [2]:
# Rodando Spark com Docker: docker run -p 8080:8080 --name pnad_covid bitnami/spark:latest
spark = SparkSession.builder.appName("PNAD COVID-19").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/03/18 19:56:50 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


## Lendo os CSVs

In [3]:
# Lendo CSVs
df_092020 = spark.read.csv("assets/PNAD_COVID_092020.csv", header=True, inferSchema=True)
df_102020 = spark.read.csv("assets/PNAD_COVID_102020.csv", header=True, inferSchema=True)
df_112020 = spark.read.csv("assets/PNAD_COVID_112020.csv", header=True, inferSchema=True)

df_092020 = df_092020.withColumnRenamed("V1013", "Mês")
df_102020 = df_102020.withColumnRenamed("V1013", "Mês")
df_112020 = df_112020.withColumnRenamed("V1013", "Mês")

## Realizando o merge de todos os dataframes dos meses 09, 10 e 11 de 2020

Nesse caso, não é possível realizar o merge dos dados considerando que há colunas existentes em uns e inexistentes em outros. Estou mapeando essas colunas, adicionando-as nos dataframes que não as possuem e configurando o valor para None.

In [4]:
# Identificando todas as colunas únicas em todos os DataFrames
colunas_df1 = set(df_112020.columns)
colunas_df2 = set(df_102020.columns)
colunas_df3 = set(df_092020.columns)

todas_colunas = colunas_df1.union(colunas_df2).union(colunas_df3)

# Etapa 2: Adicionar colunas que estão faltando em cada DataFrame
def adicionar_colunas_faltantes(df, colunas):
    colunas_faltantes = colunas - set(df.columns)
    for coluna in colunas_faltantes:
        df = df.withColumn(coluna, lit(None))
    return df

df_112020 = adicionar_colunas_faltantes(df_112020, todas_colunas)
df_102020 = adicionar_colunas_faltantes(df_102020, todas_colunas)
df_092020 = adicionar_colunas_faltantes(df_092020, todas_colunas)

# Realizando o merge dos DataFrames
df_merged = df_112020.unionByName(df_102020).unionByName(df_092020)

df_merged.printSchema()

root
 |-- Ano: integer (nullable = true)
 |-- UF: integer (nullable = true)
 |-- CAPITAL: integer (nullable = true)
 |-- RM_RIDE: integer (nullable = true)
 |-- V1008: integer (nullable = true)
 |-- V1012: integer (nullable = true)
 |-- Mês: integer (nullable = true)
 |-- V1016: integer (nullable = true)
 |-- Estrato: integer (nullable = true)
 |-- UPA: integer (nullable = true)
 |-- V1022: integer (nullable = true)
 |-- V1023: integer (nullable = true)
 |-- V1030: integer (nullable = true)
 |-- V1031: double (nullable = true)
 |-- V1032: double (nullable = true)
 |-- posest: integer (nullable = true)
 |-- A001: integer (nullable = true)
 |-- A001A: integer (nullable = true)
 |-- A001B1: integer (nullable = true)
 |-- A001B2: integer (nullable = true)
 |-- A001B3: integer (nullable = true)
 |-- A002: integer (nullable = true)
 |-- A003: integer (nullable = true)
 |-- A004: integer (nullable = true)
 |-- A005: integer (nullable = true)
 |-- A006: integer (nullable = true)
 |-- A006A: in

## Escolhendo as colunas que serão utilizadas
1. Características dos Moradores:
- A002: Idade;
- A003: Sexo;
- A004: Cor ou Raça;
- A005: Escolaridade;

2. Sintomas:
- B0011: Na semana passada teve febre?
- B0012: Na semana passada teve tosse?
- B0013: Na semana passada teve dor de garganta?
- B0014: Na semana passada teve dificuldade para respirar?
- B0015: Na semana passada teve dor de cabeça?
- B0016: Na semana passada teve dor no peito?
- B0017: Na semana passada teve náusea?
- B0018: Na semana passada teve nariz entupido ou escorrendo?
- B0019: Na semana passada teve fadiga?
- B00110: Na semana passada teve dor nos olhos?
- B00111: Na semana passada teve perda de cheiro ou sabor?
- B00112: Na semana passada teve dor muscular?

3. Características de trabalho das pessoas de 14 anos ou mais de idade:
- C001: Na semana passada, por pelo menos uma hora, trabalhou ou fez algum bico?
- C002: Na semana passada, estava temporariamente afastado de algum trabalho?
- C005: Há quanto tempo está afastado desse trabalho?

## Removendo as colunas restantes

In [5]:
df_merged = df_merged.drop("RM_RIDE","V1008","V1012","V1016","Estrato","UPA","V1022","V1023","V1030","V1031","V1032","posest")
df_merged = df_merged.drop("CAPITAL", "A006", "A006A", "A006B", "A007", "A007A", "A008", "A009", "B00113", "B008", "B009A", "B009B", "B009C", "B009D", "B009E", "B009F", "B0101", "B0102", "B0103", "B0104", "B0105", "B0106", "B011", "C007F", "C009A", "E001", "E0021", "E0022", "E0023", "E0024", "F002A1", "F002A2", "F002A3", "F002A4", "F002A5")
df_merged = df_merged.drop("A001","A001A","A001B1","A001B2","A001B3")
df_merged = df_merged.drop("B002","B0031","B0032","B0033","B0034","B0035","B0036","B0037","B0041","B0042","B0043","B0044","B0045","B0046","B005","B006","B007")
df_merged = df_merged.drop("C003","C004","C0051","C0052","C0053","C006","C007","C007A","C007B","C007C","C007D","C007E","C007E1","C007E2","C008","C009","C010","C0101","C01011","C01012","C0102","C01021","C01022","C0103","C0104","C011A","C011A1","C011A11","C011A12","C011A2","C011A21","C011A22","C012","C013","C014","C015","C016","C017A")
df_merged = df_merged.drop("D0011","D0013","D0021","D0023","D0031","D0033","D0041","D0043","D0051","D0053","D0061","D0063","D0071","D0073","F001","F0021","F0022","F0061","F006")

df_merged.printSchema()

root
 |-- Ano: integer (nullable = true)
 |-- UF: integer (nullable = true)
 |-- Mês: integer (nullable = true)
 |-- A002: integer (nullable = true)
 |-- A003: integer (nullable = true)
 |-- A004: integer (nullable = true)
 |-- A005: integer (nullable = true)
 |-- B0011: integer (nullable = true)
 |-- B0012: integer (nullable = true)
 |-- B0013: integer (nullable = true)
 |-- B0014: integer (nullable = true)
 |-- B0015: integer (nullable = true)
 |-- B0016: integer (nullable = true)
 |-- B0017: integer (nullable = true)
 |-- B0018: integer (nullable = true)
 |-- B0019: integer (nullable = true)
 |-- B00110: integer (nullable = true)
 |-- B00111: integer (nullable = true)
 |-- B00112: integer (nullable = true)
 |-- C001: integer (nullable = true)
 |-- C002: integer (nullable = true)
 |-- C005: integer (nullable = true)



## Atualizando o Dataframe com o mapeamento do dicionário do PNAD COVID-19

In [6]:
# Atualizando UF
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "11", "Rondônia").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "12", "Acre").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "13", "Amazonas").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "14", "Roraima").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "15", "Pará").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "16", "Amapá").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "17", "Tocantins").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "21", "Maranhão").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "22", "Piauí").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "23", "Ceará").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "24", "Rio Grande do Norte").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "25", "Paraíba").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "26", "Pernambuco").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "27", "Alagoas").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "28", "Sergipe").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "29", "Bahia").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "31", "Minas Gerais").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "32", "Espírito Santo").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "33", "Rio de Janeiro").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "35", "São Paulo").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "41", "Paraná").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "42", "Santa Catarina").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "43", "Rio Grande do Sul").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "50", "Mato Grosso do Sul").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "51", "Mato Grosso").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "52", "Goiás").otherwise(df_merged["UF"]))
df_merged = df_merged.withColumn("UF", when(df_merged["UF"] == "53", "Distrito Federal").otherwise(df_merged["UF"]))

In [7]:
# Atualizando idade
df_merged = df_merged.withColumnRenamed("A002", "Idade")

In [8]:
# Atualizando sexo
df_merged = df_merged.withColumnRenamed("A003", "Sexo")

df_merged = df_merged.withColumn("Sexo", when(df_merged["Sexo"] == "1", "Homem").otherwise(df_merged["Sexo"]))
df_merged = df_merged.withColumn("Sexo", when(df_merged["Sexo"] == "2", "Mulher").otherwise(df_merged["Sexo"]))

In [9]:
# Atualizando Cor/Raça
df_merged = df_merged.withColumnRenamed("A004", "Cor ou Raça")

df_merged = df_merged.withColumn("Cor ou Raça", when(df_merged["Cor ou Raça"] == "1", "Branca").otherwise(df_merged["Cor ou Raça"]))
df_merged = df_merged.withColumn("Cor ou Raça", when(df_merged["Cor ou Raça"] == "2", "Preta").otherwise(df_merged["Cor ou Raça"]))
df_merged = df_merged.withColumn("Cor ou Raça", when(df_merged["Cor ou Raça"] == "3", "Amarela").otherwise(df_merged["Cor ou Raça"]))
df_merged = df_merged.withColumn("Cor ou Raça", when(df_merged["Cor ou Raça"] == "4", "Parda").otherwise(df_merged["Cor ou Raça"]))
df_merged = df_merged.withColumn("Cor ou Raça", when(df_merged["Cor ou Raça"] == "5", "Indígena").otherwise(df_merged["Cor ou Raça"]))
df_merged = df_merged.withColumn("Cor ou Raça", when(df_merged["Cor ou Raça"] == "9", "Ignorado").otherwise(df_merged["Cor ou Raça"]))

In [10]:
# Atualizando Escolaridade
df_merged = df_merged.withColumnRenamed("A005", "Escolaridade")

df_merged = df_merged.withColumn("Escolaridade", when(df_merged["Escolaridade"] == "1", "Sem instrução").otherwise(df_merged["Escolaridade"]))
df_merged = df_merged.withColumn("Escolaridade", when(df_merged["Escolaridade"] == "2", "Fundamental incompleto").otherwise(df_merged["Escolaridade"]))
df_merged = df_merged.withColumn("Escolaridade", when(df_merged["Escolaridade"] == "3", "Fundamental completa").otherwise(df_merged["Escolaridade"]))
df_merged = df_merged.withColumn("Escolaridade", when(df_merged["Escolaridade"] == "4", "Médio incompleto").otherwise(df_merged["Escolaridade"]))
df_merged = df_merged.withColumn("Escolaridade", when(df_merged["Escolaridade"] == "5", "Médio completo").otherwise(df_merged["Escolaridade"]))
df_merged = df_merged.withColumn("Escolaridade", when(df_merged["Escolaridade"] == "6", "Superior incompleto").otherwise(df_merged["Escolaridade"]))
df_merged = df_merged.withColumn("Escolaridade", when(df_merged["Escolaridade"] == "7", "Superior completo").otherwise(df_merged["Escolaridade"]))
df_merged = df_merged.withColumn("Escolaridade", when(df_merged["Escolaridade"] == "8", "Pós-graduação, mestrado ou doutorado").otherwise(df_merged["Escolaridade"]))

In [11]:
# Atualizando sintomas
df_merged = df_merged.withColumnRenamed("B0011", "Febre")
df_merged = df_merged.withColumn("Febre", when(df_merged["Febre"] == "1", "Sim").otherwise(df_merged["Febre"]))
df_merged = df_merged.withColumn("Febre", when(df_merged["Febre"] == "2", "Não").otherwise(df_merged["Febre"]))
df_merged = df_merged.withColumn("Febre", when(df_merged["Febre"] == "3", "Não sabe").otherwise(df_merged["Febre"]))
df_merged = df_merged.withColumn("Febre", when(df_merged["Febre"] == "9", "Ignorado").otherwise(df_merged["Febre"]))

df_merged = df_merged.withColumnRenamed("B0012", "Tosse")
df_merged = df_merged.withColumn("Tosse", when(df_merged["Tosse"] == "1", "Sim").otherwise(df_merged["Tosse"]))
df_merged = df_merged.withColumn("Tosse", when(df_merged["Tosse"] == "2", "Não").otherwise(df_merged["Tosse"]))
df_merged = df_merged.withColumn("Tosse", when(df_merged["Tosse"] == "3", "Não sabe").otherwise(df_merged["Tosse"]))
df_merged = df_merged.withColumn("Tosse", when(df_merged["Tosse"] == "9", "Ignorado").otherwise(df_merged["Tosse"]))

df_merged = df_merged.withColumnRenamed("B0013", "Dor de Garganta")
df_merged = df_merged.withColumn("Dor de Garganta", when(df_merged["Dor de Garganta"] == "1", "Sim").otherwise(df_merged["Dor de Garganta"]))
df_merged = df_merged.withColumn("Dor de Garganta", when(df_merged["Dor de Garganta"] == "2", "Não").otherwise(df_merged["Dor de Garganta"]))
df_merged = df_merged.withColumn("Dor de Garganta", when(df_merged["Dor de Garganta"] == "3", "Não sabe").otherwise(df_merged["Dor de Garganta"]))
df_merged = df_merged.withColumn("Dor de Garganta", when(df_merged["Dor de Garganta"] == "9", "Ignorado").otherwise(df_merged["Dor de Garganta"]))

df_merged = df_merged.withColumnRenamed("B0014", "Dificuldade para Respirar")
df_merged = df_merged.withColumn("Dificuldade para Respirar", when(df_merged["Dificuldade para Respirar"] == "1", "Sim").otherwise(df_merged["Dificuldade para Respirar"]))
df_merged = df_merged.withColumn("Dificuldade para Respirar", when(df_merged["Dificuldade para Respirar"] == "2", "Não").otherwise(df_merged["Dificuldade para Respirar"]))
df_merged = df_merged.withColumn("Dificuldade para Respirar", when(df_merged["Dificuldade para Respirar"] == "3", "Não sabe").otherwise(df_merged["Dificuldade para Respirar"]))
df_merged = df_merged.withColumn("Dificuldade para Respirar", when(df_merged["Dificuldade para Respirar"] == "9", "Ignorado").otherwise(df_merged["Dificuldade para Respirar"]))

df_merged = df_merged.withColumnRenamed("B0015", "Dor de Cabeça")
df_merged = df_merged.withColumn("Dor de Cabeça", when(df_merged["Dor de Cabeça"] == "1", "Sim").otherwise(df_merged["Dor de Cabeça"]))
df_merged = df_merged.withColumn("Dor de Cabeça", when(df_merged["Dor de Cabeça"] == "2", "Não").otherwise(df_merged["Dor de Cabeça"]))
df_merged = df_merged.withColumn("Dor de Cabeça", when(df_merged["Dor de Cabeça"] == "3", "Não sabe").otherwise(df_merged["Dor de Cabeça"]))
df_merged = df_merged.withColumn("Dor de Cabeça", when(df_merged["Dor de Cabeça"] == "9", "Ignorado").otherwise(df_merged["Dor de Cabeça"]))

df_merged = df_merged.withColumnRenamed("B0016", "Dor no Peito")
df_merged = df_merged.withColumn("Dor no Peito", when(df_merged["Dor no Peito"] == "1", "Sim").otherwise(df_merged["Dor no Peito"]))
df_merged = df_merged.withColumn("Dor no Peito", when(df_merged["Dor no Peito"] == "2", "Não").otherwise(df_merged["Dor no Peito"]))
df_merged = df_merged.withColumn("Dor no Peito", when(df_merged["Dor no Peito"] == "3", "Não sabe").otherwise(df_merged["Dor no Peito"]))
df_merged = df_merged.withColumn("Dor no Peito", when(df_merged["Dor no Peito"] == "9", "Ignorado").otherwise(df_merged["Dor no Peito"]))

df_merged = df_merged.withColumnRenamed("B0017", "Náusea")
df_merged = df_merged.withColumn("Náusea", when(df_merged["Náusea"] == "1", "Sim").otherwise(df_merged["Náusea"]))
df_merged = df_merged.withColumn("Náusea", when(df_merged["Náusea"] == "2", "Não").otherwise(df_merged["Náusea"]))
df_merged = df_merged.withColumn("Náusea", when(df_merged["Náusea"] == "3", "Não sabe").otherwise(df_merged["Náusea"]))
df_merged = df_merged.withColumn("Náusea", when(df_merged["Náusea"] == "9", "Ignorado").otherwise(df_merged["Náusea"]))

df_merged = df_merged.withColumnRenamed("B0018", "Nariz Entupido ou Escorrendo")
df_merged = df_merged.withColumn("Nariz Entupido ou Escorrendo", when(df_merged["Nariz Entupido ou Escorrendo"] == "1", "Sim").otherwise(df_merged["Nariz Entupido ou Escorrendo"]))
df_merged = df_merged.withColumn("Nariz Entupido ou Escorrendo", when(df_merged["Nariz Entupido ou Escorrendo"] == "2", "Não").otherwise(df_merged["Nariz Entupido ou Escorrendo"]))
df_merged = df_merged.withColumn("Nariz Entupido ou Escorrendo", when(df_merged["Nariz Entupido ou Escorrendo"] == "3", "Não sabe").otherwise(df_merged["Nariz Entupido ou Escorrendo"]))
df_merged = df_merged.withColumn("Nariz Entupido ou Escorrendo", when(df_merged["Nariz Entupido ou Escorrendo"] == "9", "Ignorado").otherwise(df_merged["Nariz Entupido ou Escorrendo"]))

df_merged = df_merged.withColumnRenamed("B0019", "Fadiga")
df_merged = df_merged.withColumn("Fadiga", when(df_merged["Fadiga"] == "1", "Sim").otherwise(df_merged["Fadiga"]))
df_merged = df_merged.withColumn("Fadiga", when(df_merged["Fadiga"] == "2", "Não").otherwise(df_merged["Fadiga"]))
df_merged = df_merged.withColumn("Fadiga", when(df_merged["Fadiga"] == "3", "Não sabe").otherwise(df_merged["Fadiga"]))
df_merged = df_merged.withColumn("Fadiga", when(df_merged["Fadiga"] == "9", "Ignorado").otherwise(df_merged["Fadiga"]))

df_merged = df_merged.withColumnRenamed("B00110", "Dor nos Olhos")
df_merged = df_merged.withColumn("Dor nos Olhos", when(df_merged["Dor nos Olhos"] == "1", "Sim").otherwise(df_merged["Dor nos Olhos"]))
df_merged = df_merged.withColumn("Dor nos Olhos", when(df_merged["Dor nos Olhos"] == "2", "Não").otherwise(df_merged["Dor nos Olhos"]))
df_merged = df_merged.withColumn("Dor nos Olhos", when(df_merged["Dor nos Olhos"] == "3", "Não sabe").otherwise(df_merged["Dor nos Olhos"]))
df_merged = df_merged.withColumn("Dor nos Olhos", when(df_merged["Dor nos Olhos"] == "9", "Ignorado").otherwise(df_merged["Dor nos Olhos"]))

df_merged = df_merged.withColumnRenamed("B00111", "Perda de Cheiro ou Sabor")
df_merged = df_merged.withColumn("Perda de Cheiro ou Sabor", when(df_merged["Perda de Cheiro ou Sabor"] == "1", "Sim").otherwise(df_merged["Perda de Cheiro ou Sabor"]))
df_merged = df_merged.withColumn("Perda de Cheiro ou Sabor", when(df_merged["Perda de Cheiro ou Sabor"] == "2", "Não").otherwise(df_merged["Perda de Cheiro ou Sabor"]))
df_merged = df_merged.withColumn("Perda de Cheiro ou Sabor", when(df_merged["Perda de Cheiro ou Sabor"] == "3", "Não sabe").otherwise(df_merged["Perda de Cheiro ou Sabor"]))
df_merged = df_merged.withColumn("Perda de Cheiro ou Sabor", when(df_merged["Perda de Cheiro ou Sabor"] == "9", "Ignorado").otherwise(df_merged["Perda de Cheiro ou Sabor"]))

df_merged = df_merged.withColumnRenamed("B00112", "Dor Muscular")
df_merged = df_merged.withColumn("Dor Muscular", when(df_merged["Dor Muscular"] == "1", "Sim").otherwise(df_merged["Dor Muscular"]))
df_merged = df_merged.withColumn("Dor Muscular", when(df_merged["Dor Muscular"] == "2", "Não").otherwise(df_merged["Dor Muscular"]))
df_merged = df_merged.withColumn("Dor Muscular", when(df_merged["Dor Muscular"] == "3", "Não sabe").otherwise(df_merged["Dor Muscular"]))
df_merged = df_merged.withColumn("Dor Muscular", when(df_merged["Dor Muscular"] == "9", "Ignorado").otherwise(df_merged["Dor Muscular"]))

In [12]:
# Atualizando comportamento no trabalho
df_merged = df_merged.withColumnRenamed("C001", "Trabalhou")
df_merged = df_merged.withColumn("Trabalhou", when(df_merged["Trabalhou"] == "1", "Sim").otherwise(df_merged["Trabalhou"]))
df_merged = df_merged.withColumn("Trabalhou", when(df_merged["Trabalhou"] == "2", "Não").otherwise(df_merged["Trabalhou"]))
df_merged = df_merged.withColumn("Trabalhou", when((df_merged["Trabalhou"] != 1) & (df_merged["Trabalhou"] != 2), "Não aplicável").otherwise(df_merged["Trabalhou"]))

df_merged = df_merged.withColumnRenamed("C002", "Afastado")
df_merged = df_merged.withColumn("Afastado", when(df_merged["Afastado"] == "1", "Sim").otherwise(df_merged["Afastado"]))
df_merged = df_merged.withColumn("Afastado", when(df_merged["Afastado"] == "2", "Não").otherwise(df_merged["Afastado"]))
df_merged = df_merged.withColumn("Afastado", when((df_merged["Afastado"] != 1) & (df_merged["Afastado"] != 2), "Não aplicável").otherwise(df_merged["Afastado"]))


df_merged = df_merged.withColumnRenamed("C005", "Tempo Afastado")
df_merged = df_merged.withColumn("Tempo Afastado", when(df_merged["Tempo Afastado"] == "1", "Menos de 1 mês").otherwise(df_merged["Tempo Afastado"]))
df_merged = df_merged.withColumn("Tempo Afastado", when(df_merged["Tempo Afastado"] == "2", "De 1 mês a menos de 1 ano").otherwise(df_merged["Tempo Afastado"]))
df_merged = df_merged.withColumn("Tempo Afastado", when(df_merged["Tempo Afastado"] == "3", "De 1 ano a menos de 2 anos").otherwise(df_merged["Tempo Afastado"]))
df_merged = df_merged.withColumn("Tempo Afastado", when(df_merged["Tempo Afastado"] == "4", "2 anos ou mais").otherwise(df_merged["Tempo Afastado"]))
df_merged = df_merged.withColumn("Tempo Afastado", when((df_merged["Tempo Afastado"] != 1) & (df_merged["Tempo Afastado"] != 2) & (df_merged["Tempo Afastado"] != 3) & (df_merged["Tempo Afastado"] != 4), "Não aplicável").otherwise(df_merged["Tempo Afastado"]))

In [13]:
df_merged.toPandas().sample(10)

24/03/18 19:57:02 WARN GarbageCollectionMetrics: To enable non-built-in garbage collector(s) List(G1 Concurrent GC), users should configure it(them) to spark.eventLog.gcMetrics.youngGenerationGarbageCollectors or spark.eventLog.gcMetrics.oldGenerationGarbageCollectors


,Ano,UF,Mês,Idade,Sexo,Cor ou Raça,Escolaridade,Febre,Tosse,Dor de Garganta,...,Dor no Peito,Náusea,Nariz Entupido ou Escorrendo,Fadiga,Dor nos Olhos,Perda de Cheiro ou Sabor,Dor Muscular,Trabalhou,Afastado,Tempo Afastado
331996,2020,Rio Grande do Sul,11,30,Mulher,Branca,Médio completo,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,None
318129,2020,Santa Catarina,11,23,Homem,Branca,Fundamental completa,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Sim,None,None
565439,2020,Minas Gerais,10,44,Mulher,Branca,Fundamental incompleto,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,None
503003,2020,Pernambuco,10,55,Mulher,Parda,"Pós-graduação, mestrado ou doutorado",Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,None
665456,2020,Paraná,10,49,Homem,Branca,Médio completo,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Sim,None,None
511313,2020,Alagoas,10,32,Homem,Parda,Fundamental incompleto,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,None
220735,2020,Rio de Janeiro,11,22,Homem,Parda,Médio completo,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,None
137188,2020,Alagoas,11,58,Homem,Branca,Fundamental incompleto,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,None
389202,2020,Acre,10,35,Homem,Preta,Médio completo,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Sim,None,None
1128451,2020,Mato Grosso,9,28,Homem,Parda,Sem instrução,Não,Não,Não,...,Não,Não,Não,Não,Não,Não,Não,Não,Não,None


## Algumas possíveis análises
- Escolaridade/Idade/Sexo x UF;
- As pessoas infectadas continuaram trabalhando?
- 10 sintomas mais comuns;